In [1]:
import numpy as np             
import pandas as pd             
import os                       
import matplotlib.pyplot as plt 
from IPython.display import display
from random import seed

/tmp/ipykernel_2466/2323600563.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
financement = pd.read_csv(filepath_or_buffer='financement.csv', sep=',', header=0, index_col=0)

financement.head(5)

,ID,nom,categorie,categorie_principale,monaie,date_butoire,objectif,lancement,promesse,etat,supporters,pays,promesse_usd1,promesse_usd2,objectif_usd
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [11]:
#fonction detail dataframe
def resume_dataframe(df):
    resume_df = pd.DataFrame(index=df.columns)
    resume_df['Type'] = df.dtypes.values
    resume_df['Missing'] = df.isnull().sum().values

    resume_df['Unique'] = df.nunique().values

    for col in df.columns:
        resume_df.loc[col, 'Value_1'] = df[col].iloc[0]
        resume_df.loc[col, 'Value_2'] = df[col].iloc[1]
        resume_df.loc[col, 'Value_3'] = df[col].iloc[2]

    return resume_df

resume_transactions = resume_dataframe(financement)

display(resume_transactions)

/tmp/ipykernel_2466/1038085726.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'The Songs of Adelaide & Abullah' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resume_df.loc[col, 'Value_1'] = df[col].iloc[0]
/tmp/ipykernel_2466/1038085726.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Greeting From Earth: ZGAC Arts Capsule For ET' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resume_df.loc[col, 'Value_2'] = df[col].iloc[1]
/tmp/ipykernel_2466/1038085726.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Where is Hank?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  resume_df.loc[col, 'Value_3'] = df

,Type,Missing,Unique,Value_1,Value_2,Value_3
ID,int64,0,378661,1000002330.0,1000003930.0,1000004038.0
nom,object,4,375764,The Songs of Adelaide & Abullah,Greeting From Earth: ZGAC Arts Capsule For ET,Where is Hank?
categorie,object,0,159,Poetry,Narrative Film,Narrative Film
categorie_principale,object,0,15,Publishing,Film & Video,Film & Video
monaie,object,0,14,GBP,USD,USD
date_butoire,object,0,3164,2015-10-09,2017-11-01,2013-02-26
objectif,float64,0,8353,1000.0,30000.0,45000.0
lancement,object,0,378089,2015-08-11 12:12:28,2017-09-02 04:43:57,2013-01-12 00:20:50
promesse,float64,0,62130,0.0,2421.0,220.0
etat,object,0,6,failed,failed,failed
